In [25]:
import pandas as pd
import json
import os


from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [26]:
df = pd.read_csv('../data/api_data.csv').drop(columns = 'Unnamed: 0')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   text                100 non-null    object
 1   contents            100 non-null    object
 2   relationships_data  100 non-null    object
 3   ERD_data            100 non-null    object
 4   API_specs_data      77 non-null     object
dtypes: object(5)
memory usage: 4.0+ KB


In [27]:
df_isna = df[df['API_specs_data'].isna()]
df_isna.head()

,text,contents,relationships_data,ERD_data,API_specs_data
5,이 웹사이트는 나만의 퍼스널 쇼퍼 역할을 하는 혁신적인 쇼핑몰입니다. 사용자의 얼굴...,"{'project_summary': {'title': '퍼스널 쇼퍼 쇼핑몰', 'c...","[{'from': 'User', 'to': 'Recommendation', 'typ...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN
7,"이 어플은 취준생들을 위한 종합 커뮤니티 플랫폼으로, 합격자들의 다양한 경험담과 실...",{'project_summary': {'title': '취준생 종합 커뮤니티 플랫폼...,"[{'from': 'User', 'to': 'Experience', 'type': ...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN
9,이 웹앱 프로젝트는 한국어 학습자들을 위한 통합 학습 플랫폼입니다. 사용자는 일반 ...,"{'project_summary': {'title': '한국어 학습 통합 플랫폼',...","[{'from': 'User', 'to': 'Translation', 'type':...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN
11,"스마트 러닝 메이트는 AI 챗봇과 함께 자기주도적 학습을 체계적으로 관리하고, 집중...","{'project_summary': {'title': '스마트 러닝 메이트', 'c...","[{'from': 'User', 'to': 'LearningGoal', 'type'...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN
18,"*머니메이트(MoneyMate)**는 사용자의 소비 패턴을 분석하고, 맞춤형 금융상...",{'project_summary': {'title': '머니메이트(MoneyMate...,"[{'from': 'User', 'to': 'Transaction', 'type':...","[{'name': 'User', 'attributes': [{'name': 'use...",NaN


In [24]:
load_dotenv()

api_key = os.getenv('API_KEY')

In [ ]:
API_lst = []

sys_template = """
당신은 사용자가 입력한 프로젝트 개요를 기반으로 예상되는 API 명세서를 작성해주는 프로그램입니다.
주제에 대해서 예상되는 RESTful API 명세서를 JSON 형식으로 답변해주세요.

중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

예시는 다음과 같습니다.
{{
"api_specification": {{
    "openapi": "3.0.0",
    "info": {{
    "title": "User Management API",
    "version": "1.0.0",
    "description": "사용자 관리를 위한 API"
    }},
    "servers": [
    {{
        "url": "https://api.example.com/v1",
        "description": "Production server"
    }}
    ],
    "paths": {{
    "/users": {{
        "get": {{
        "summary": "사용자 목록 조회",
        "description": "등록된 모든 사용자 목록을 조회합니다",
        "tags": ["Users"],
        "parameters": [
            {{
            "name": "page",
            "in": "query",
            "description": "페이지 번호",
            "required": false,
            "schema": {{
                "type": "integer",
                "default": 1
            }}
            }},
            {{
            "name": "limit",
            "in": "query", 
            "description": "페이지당 항목 수",
            "required": false,
            "schema": {{
                "type": "integer",
                "default": 10
            }}
            }}
        ],
        "responses": {{
            "200": {{
            "description": "사용자 목록 조회 성공",
            "content": {{
                "application/json": {{
                "schema": {{
                    "type": "object",
                    "properties": {{
                    "status": {{
                        "type": "string",
                        "example": "success"
                    }},
                    "data": {{
                        "type": "array",
                        "items": {{
                        "$ref": "#/components/schemas/User"
                        }}
                    }},
                    "pagination": {{
                        "$ref": "#/components/schemas/Pagination"
                    }}
                    }}
                }}
                }}
            }}
            }},
            "400": {{
            "description": "잘못된 요청",
            "content": {{
                "application/json": {{
                "schema": {{
                    "$ref": "#/components/schemas/Error"
                }}
                }}
            }}
            }}
        }}
        }},
        "post": {{
        "summary": "새 사용자 생성",
        "description": "새로운 사용자를 생성합니다",
        "tags": ["Users"],
        "requestBody": {{
            "required": true,
            "content": {{
            "application/json": {{
                "schema": {{
                "$ref": "#/components/schemas/CreateUserRequest"
                }}
            }}
            }}
        }},
        "responses": {{
            "201": {{
            "description": "사용자 생성 성공",
            "content": {{
                "application/json": {{
                "schema": {{
                    "type": "object",
                    "properties": {{
                    "status": {{
                        "type": "string",
                        "example": "success"
                    }},
                    "data": {{
                        "$ref": "#/components/schemas/User"
                    }}
                    }}
                }}
                }}
            }}
            }},
            "400": {{
            "description": "잘못된 요청 데이터",
            "content": {{
                "application/json": {{
                "schema": {{
                    "$ref": "#/components/schemas/Error"
                }}
                }}
            }}
            }}
        }}
        }}
    }},
    "/users/{{userId}}": {{
        "get": {{
        "summary": "특정 사용자 조회",
        "description": "사용자 ID로 특정 사용자 정보를 조회합니다",
        "tags": ["Users"],
        "parameters": [
            {{
            "name": "userId",
            "in": "path",
            "required": true,
            "description": "사용자 ID",
            "schema": {{
                "type": "integer"
            }}
            }}
        ],
        "responses": {{
            "200": {{
            "description": "사용자 조회 성공",
            "content": {{
                "application/json": {{
                "schema": {{
                    "type": "object",
                    "properties": {{
                    "status": {{
                        "type": "string",
                        "example": "success"
                    }},
                    "data": {{
                        "$ref": "#/components/schemas/User"
                    }}
                    }}
                }}
                }}
            }}
            }},
            "404": {{
            "description": "사용자를 찾을 수 없음",
            "content": {{
                "application/json": {{
                "schema": {{
                    "$ref": "#/components/schemas/Error"
                }}
                }}
            }}
            }}
        }}
        }},
        "put": {{
        "summary": "사용자 정보 수정",
        "description": "사용자 정보를 수정합니다",
        "tags": ["Users"],
        "parameters": [
            {{
            "name": "userId",
            "in": "path",
            "required": true,
            "description": "사용자 ID",
            "schema": {{
                "type": "integer"
            }}
            }}
        ],
        "requestBody": {{
            "required": true,
            "content": {{
            "application/json": {{
                "schema": {{
                "$ref": "#/components/schemas/UpdateUserRequest"
                }}
            }}
            }}
        }},
        "responses": {{
            "200": {{
            "description": "사용자 정보 수정 성공",
            "content": {{
                "application/json": {{
                "schema": {{
                    "type": "object",
                    "properties": {{
                    "status": {{
                        "type": "string",
                        "example": "success"
                    }},
                    "data": {{
                        "$ref": "#/components/schemas/User"
                    }}
                    }}
                }}
                }}
            }}
            }},
            "404": {{
            "description": "사용자를 찾을 수 없음",
            "content": {{
                "application/json": {{
                "schema": {{
                    "$ref": "#/components/schemas/Error"
                }}
                }}
            }}
            }}
        }}
        }},
        "delete": {{
        "summary": "사용자 삭제",
        "description": "사용자를 삭제합니다",
        "tags": ["Users"],
        "parameters": [
            {{
            "name": "userId",
            "in": "path",
            "required": true,
            "description": "사용자 ID",
            "schema": {{
                "type": "integer"
            }}
            }}
        ],
        "responses": {{
            "200": {{
            "description": "사용자 삭제 성공",
            "content": {{
                "application/json": {{
                "schema": {{
                    "type": "object",
                    "properties": {{
                    "status": {{
                        "type": "string",
                        "example": "success"
                    }},
                    "message": {{
                        "type": "string",
                        "example": "사용자가 성공적으로 삭제되었습니다"
                    }}
                    }}
                }}
                }}
            }}
            }},
            "404": {{
            "description": "사용자를 찾을 수 없음",
            "content": {{
                "application/json": {{
                "schema": {{
                    "$ref": "#/components/schemas/Error"
                }}
                }}
            }}
            }}
        }}
        }}
    }}
    }},
    "components": {{
    "schemas": {{
        "User": {{
        "type": "object",
        "properties": {{
            "id": {{
            "type": "integer",
            "example": 1
            }},
            "username": {{
            "type": "string",
            "example": "john_doe"
            }},
            "email": {{
            "type": "string",
            "format": "email",
            "example": "john@example.com"
            }},
            "createdAt": {{
            "type": "string",
            "format": "date-time",
            "example": "2024-01-15T10:30:00Z"
            }},
            "updatedAt": {{
            "type": "string",
            "format": "date-time",
            "example": "2024-01-15T10:30:00Z"
            }}
        }}
        }},
        "CreateUserRequest": {{
        "type": "object",
        "required": ["username", "email"],
        "properties": {{
            "username": {{
            "type": "string",
            "minLength": 3,
            "maxLength": 50,
            "example": "john_doe"
            }},
            "email": {{
            "type": "string",
            "format": "email",
            "example": "john@example.com"
            }},
            "password": {{
            "type": "string",
            "minLength": 8,
            "example": "password123"
            }}
        }}
        }},
        "UpdateUserRequest": {{
        "type": "object",
        "properties": {{
            "username": {{
            "type": "string",
            "minLength": 3,
            "maxLength": 50,
            "example": "john_doe_updated"
            }},
            "email": {{
            "type": "string",
            "format": "email",
            "example": "john_updated@example.com"
            }}
        }}
        }},
        "Pagination": {{
        "type": "object",
        "properties": {{
            "currentPage": {{
            "type": "integer",
            "example": 1
            }},
            "totalPages": {{
            "type": "integer",
            "example": 10
            }},
            "totalItems": {{
            "type": "integer",
            "example": 100
            }},
            "itemsPerPage": {{
            "type": "integer",
            "example": 10
            }}
        }}
        }},
        "Error": {{
        "type": "object",
        "properties": {{
            "status": {{
            "type": "string",
            "example": "error"
            }},
            "message": {{
            "type": "string",
            "example": "오류가 발생했습니다"
            }},
            "code": {{
            "type": "string",
            "example": "VALIDATION_ERROR"
            }}
        }}
        }}
    }},
    "securitySchemes": {{
        "bearerAuth": {{
        "type": "http",
        "scheme": "bearer",
        "bearerFormat": "JWT"
        }}
    }}
    }},
    "security": [
    {{
        "bearerAuth": []
    }}
    ]
}}
}}

각 API 엔드포인트의 상세 정보(HTTP 메서드, 경로, 요청/응답 스키마, 상태 코드)를 모두 포함하세요.
프로젝트 특성에 맞는 추가적인 엔드포인트나 기능이 필요하면 포함하여 작성하세요.
JSON 형식을 정확히 지키고, 모든 속성명과 문자열 값에 쌍따옴표를 사용하세요.
OpenAPI 3.0 표준을 따르며, 실제 구현 가능한 수준의 상세한 명세서를 작성하세요.
다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
"""
user_template = "{question}"

prompt = ChatPromptTemplate.from_messages([
  ("system", sys_template),
  ("user", user_template)
])

llm = ChatOpenAI(
    api_key=api_key,
    model='gpt-4o-mini',
    temperature=0
)

llm_chain = prompt | llm

for i in range(len(df_isna))
  text = df_isna.iloc[1,1]
  answer = llm_chain.invoke({"question": text})
  answer.content


'{\n  "api_specification": {\n    "openapi": "3.0.0",\n    "info": {\n      "title": "Memory Museum API",\n      "version": "1.0.0",\n      "description": "치매 환자와 그 가족을 위한 기억 회복 웹사이트 API"\n    },\n    "servers": [\n      {\n        "url": "https://api.memorymuseum.com/v1",\n        "description": "Production server"\n      }\n    ],\n    "paths": {\n      "/memories": {\n        "get": {\n          "summary": "사용자의 기억 목록 조회",\n          "description": "사용자가 업로드한 모든 기억(사진)을 조회합니다.",\n          "tags": ["Memories"],\n          "parameters": [\n            {\n              "name": "userId",\n              "in": "query",\n              "description": "사용자 ID",\n              "required": true,\n              "schema": {\n                "type": "integer"\n              }\n            },\n            {\n              "name": "page",\n              "in": "query",\n              "description": "페이지 번호",\n              "required": false,\n              "schema": {\n                "type": "inte